In [ ]:
# Maksym Lesyk, 614228

In [ ]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
# read datasets
comp=pd.read_csv('/content/drive/MyDrive/Accounting 2 final/comp.csv')
fees=pd.read_csv('/content/drive/MyDrive/Accounting 2 final/fees.csv')

In [ ]:
# Year column
fees['Year']=fees['AUDIT_FEE_FISCAL_YEAR_END'].str[-4:]
# Concatenated value to create a unique distinguisher
fees['yearISIN'] = fees['Year'] + fees['ISIN']
# NAICS code separate on first two digits for the ease of industry classfication
fees['Industry']=fees['NAICS_CODE_FKEY'].astype("str").str[:2]
# where network is na, substitute with values from auditor name
fees['AUDITOR_NETWORK'] = fees['AUDITOR_NETWORK'].mask(fees['AUDITOR_NETWORK'].isna(), fees['AUDITOR_NAME'])
# substitute two-digit NAICS encoding with industries per paper, sometimes based on own judgement and NAICS guidelines
codes=["11","21","22","23","31","32","33","42","44","45","48","49","51","52","53","54","55","56","61","62","71","72","81","92"]
industry=["Agriculture, Forestry, Fishing and Hunting","Mining","Transportation & Public Utilities","Construction","Manufacturing","Manufacturing","Manufacturing","Wholesale Trade","Retail Trade","Retail Trade","Transportation & Public Utilities","Transportation & Public Utilities","Services","Finance, Insurance & Real Estate","Finance, Insurance & Real Estate","Services","Services","Services","Services","Services","Services","Services","Services","Public Administration"]
fees['Industry'] = fees['Industry'].replace(codes, industry)

In [ ]:
# sometimes joint audit is present, need to control for that with the above produced concatenated variable
pivot=fees.pivot_table(index=['yearISIN'], aggfunc='size')
print(pivot.sort_values(ascending=False))

yearISIN
2017FR0010436584    13
2012FR0000064180    12
2016FR0000045072    12
2019SE0006256558    11
2015FR0000072563    11
                    ..
2012GB00BKTRF404     1
2015SE0008991335     1
2012GB00BKTCLJ25     1
2012GB00BKT6VH21     1
2016BE0003008019     1
Length: 63903, dtype: int64


In [ ]:
# produce columns for table 3 panel c based on table 2 from the paper
comp["leverage"]=comp["lt"]/comp["at"]
comp["roa"]=comp["nicon"]/comp["at"]
comp['loss'] = comp.apply(lambda x : 1 if x['nicon'] < 0 else 0, axis=1)

In [ ]:
# subset on our exchanges and HQ countries
ger=fees[fees['HEADQUARTER_COUNTRY']=="Germany"]
ger=ger[(ger["PRIMARY_EXCHANGE"] == 'XETRA') | (ger["PRIMARY_EXCHANGE"] == 'DEUTSCHE BOERSE AG')]

In [ ]:
uk=fees[fees['HEADQUARTER_COUNTRY']=="United Kingdom"]
uk=uk[uk['PRIMARY_EXCHANGE']=="London Stock Exchange"]

In [ ]:
# subset on relevant submarkets. Eliminate from sample where NA
uk_aim=uk[(uk["EXCHANGE_SUBMARKET_ID"] == 'EXTP$$$AIM') | (uk["EXCHANGE_SUBMARKET_ID"] == 'EX$$$$AIMX')]
uk_mm=uk[uk['EXCHANGE_SUBMARKET_ID']=="EXTP$$$LMM"]
ger_reg=ger[(ger["EXCHANGE_SUBMARKET_ID"] == 'EXTP$$$XPS') | (ger["EXCHANGE_SUBMARKET_ID"] == 'EXTP$$$XGS')]
ger_nreg=ger[(ger["EXCHANGE_SUBMARKET_ID"] == 'EX$$$$XETS') | (ger["EXCHANGE_SUBMARKET_ID"] == 'EX$$$$FRAB') | (ger["EXCHANGE_SUBMARKET_ID"] == 'EX$$$$XETB') | (ger["EXCHANGE_SUBMARKET_ID"] == 'EX$$$$FRAS')]

In [ ]:
# merged dataset
merged = pd.concat([uk_aim,uk_mm,ger_reg,ger_nreg])
merged = merged.drop_duplicates(subset=['yearISIN'], keep='first')
comp['yearISIN'] = comp['fyear'].dropna().astype("int").astype("str").str[:4] + comp['isin']

In [ ]:
# industries in merged
indm=merged["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
indm["Percentages"]=indm["counts"]/len(merged)*100
print(indm)

                                unique_values  counts  Percentages
0            Finance, Insurance & Real Estate    4988    32.813631
1                                    Services    3730    24.537859
2                               Manufacturing    3234    21.274916
3                                      Mining    1183     7.782383
4           Transportation & Public Utilities     571     3.756332
5                                Retail Trade     571     3.756332
6                             Wholesale Trade     430     2.828761
7                                Construction     384     2.526150
8  Agriculture, Forestry, Fishing and Hunting      88     0.578909
9                       Public Administration      22     0.144727


In [ ]:
# Check for total clients and client years in comparison with the values from the paper
# matches with the described 20 percent less if based solely on Audit Analytics, which is consistent with the paper text
obs1=len(uk_aim['yearISIN'].unique())
obs2=len(uk_mm['yearISIN'].unique())
obs3=len(ger_reg['yearISIN'].unique())
obs4=len(ger_nreg['yearISIN'].unique())
tobs=obs1+obs2+obs3+obs4
pobs=17649
# Client years
tobs/pobs

0.8612952575216726

In [ ]:
obs01=len(uk_aim['ISIN'].unique())
obs02=len(uk_mm['ISIN'].unique())
obs03=len(ger_reg['ISIN'].unique())
obs04=len(ger_nreg['ISIN'].unique())
ttobs=obs01+obs02+obs03+obs04
ppobs=2196
# Clients
ttobs/ppobs

0.8064663023679417

In [ ]:
#Table 3 Panel A
# UK year mm

mm2011=len(uk_mm[uk_mm['Year']=="2011"]["yearISIN"].unique())
mm2012=len(uk_mm[uk_mm['Year']=="2012"]["yearISIN"].unique())
mm2013=len(uk_mm[uk_mm['Year']=="2013"]["yearISIN"].unique())
mm2014=len(uk_mm[uk_mm['Year']=="2014"]["yearISIN"].unique())
mm2015=len(uk_mm[uk_mm['Year']=="2015"]["yearISIN"].unique())

mm2016=len(uk_mm[uk_mm['Year']=="2016"]["yearISIN"].unique())
mm2017=len(uk_mm[uk_mm['Year']=="2017"]["yearISIN"].unique())
mm2018=len(uk_mm[uk_mm['Year']=="2018"]["yearISIN"].unique())
mm2019=len(uk_mm[uk_mm['Year']=="2019"]["yearISIN"].unique())
mm2020=len(uk_mm[uk_mm['Year']=="2020"]["yearISIN"].unique())
# unique clients
uISINmm=len(uk_mm['ISIN'].unique())

# UK year aim
aim2011=len(uk_aim[uk_aim['Year']=="2011"]["yearISIN"].unique())
aim2012=len(uk_aim[uk_aim['Year']=="2012"]["yearISIN"].unique())
aim2013=len(uk_aim[uk_aim['Year']=="2013"]["yearISIN"].unique())
aim2014=len(uk_aim[uk_aim['Year']=="2014"]["yearISIN"].unique())
aim2015=len(uk_aim[uk_aim['Year']=="2015"]["yearISIN"].unique())

aim2016=len(uk_aim[uk_aim['Year']=="2016"]["yearISIN"].unique())
aim2017=len(uk_aim[uk_aim['Year']=="2017"]["yearISIN"].unique())
aim2018=len(uk_aim[uk_aim['Year']=="2018"]["yearISIN"].unique())
aim2019=len(uk_aim[uk_aim['Year']=="2019"]["yearISIN"].unique())
aim2020=len(uk_aim[uk_aim['Year']=="2020"]["yearISIN"].unique())
# unique clients
uISINaim=len(uk_aim['ISIN'].unique())

# Germany year ger_reg
regg2011=len(ger_reg[ger_reg['Year']=="2011"]["yearISIN"].unique())
regg2012=len(ger_reg[ger_reg['Year']=="2012"]["yearISIN"].unique())
regg2013=len(ger_reg[ger_reg['Year']=="2013"]["yearISIN"].unique())
regg2014=len(ger_reg[ger_reg['Year']=="2014"]["yearISIN"].unique())
regg2015=len(ger_reg[ger_reg['Year']=="2015"]["yearISIN"].unique())

regg2016=len(ger_reg[ger_reg['Year']=="2016"]["yearISIN"].unique())
regg2017=len(ger_reg[ger_reg['Year']=="2017"]["yearISIN"].unique())
regg2018=len(ger_reg[ger_reg['Year']=="2018"]["yearISIN"].unique())
regg2019=len(ger_reg[ger_reg['Year']=="2019"]["yearISIN"].unique())
regg2020=len(ger_reg[ger_reg['Year']=="2020"]["yearISIN"].unique())
# unique clients
uISINregg=len(ger_reg['ISIN'].unique())

#Germany year ger_nreg
nregg2011=len(ger_nreg[ger_nreg['Year']=="2011"]["yearISIN"].unique())
nregg2012=len(ger_nreg[ger_nreg['Year']=="2012"]["yearISIN"].unique())
nregg2013=len(ger_nreg[ger_nreg['Year']=="2013"]["yearISIN"].unique())
nregg2014=len(ger_nreg[ger_nreg['Year']=="2014"]["yearISIN"].unique())
nregg2015=len(ger_nreg[ger_nreg['Year']=="2015"]["yearISIN"].unique())

nregg2016=len(ger_nreg[ger_nreg['Year']=="2016"]["yearISIN"].unique())
nregg2017=len(ger_nreg[ger_nreg['Year']=="2017"]["yearISIN"].unique())
nregg2018=len(ger_nreg[ger_nreg['Year']=="2018"]["yearISIN"].unique())
nregg2019=len(ger_nreg[ger_nreg['Year']=="2019"]["yearISIN"].unique())
nregg2020=len(ger_nreg[ger_nreg['Year']=="2020"]["yearISIN"].unique())
# unique clients
uISINnregg=len(ger_nreg['ISIN'].unique())


# create a dataframe for Table 3 Panel A representation
data = [['Pre-reform','2011', mm2011+aim2011+regg2011+nregg2011, mm2011+aim2011, mm2011, aim2011, regg2011+nregg2011, regg2011, nregg2011],
        ['','2012', mm2012+aim2012+regg2012+nregg2012, mm2012+aim2012, mm2012, aim2012, regg2012+nregg2012, regg2012, nregg2012],
        ['','2013', mm2013+aim2013+regg2013+nregg2013, mm2013+aim2013, mm2013, aim2013, regg2013+nregg2013, regg2013, nregg2013],
        ['','2014', mm2014+aim2014+regg2014+nregg2014, mm2014+aim2014, mm2014, aim2014, regg2014+nregg2014, regg2014, nregg2014],
        ['','2015', mm2015+aim2015+regg2015+nregg2015, mm2015+aim2015, mm2015, aim2015, regg2015+nregg2015, regg2015, nregg2015],
        ['','Subtotal:', mm2011+aim2011+regg2011+nregg2011+mm2012+aim2012+regg2012+nregg2012+mm2013+aim2013+regg2013+nregg2013+mm2014+aim2014+regg2014+nregg2014+mm2015+aim2015+regg2015+nregg2015,
         mm2011+aim2011+mm2012+aim2012+mm2013+aim2013+mm2014+aim2014+mm2015+aim2015, mm2011+mm2012+mm2013+mm2014+mm2015,
         aim2011+aim2012+aim2013+aim2014+aim2015, regg2011+nregg2011+regg2012+nregg2012+regg2013+nregg2013+regg2014+nregg2014+regg2015+nregg2015,
         regg2011+regg2012+regg2013+regg2014+regg2015, nregg2011+nregg2012+nregg2013+nregg2014+nregg2015],
        ['Post-reform','2016', mm2016+aim2016+regg2016+nregg2016, mm2016+aim2016, mm2016, aim2016, regg2016+nregg2016, regg2016, nregg2016],
        ['','2017', mm2017+aim2017+regg2017+nregg2017, mm2017+aim2017, mm2017, aim2017, regg2017+nregg2017, regg2017, nregg2017],
        ['','2018', mm2018+aim2018+regg2018+nregg2018, mm2018+aim2018, mm2018, aim2018, regg2018+nregg2018, regg2018, nregg2018],
        ['','2019', mm2019+aim2019+regg2019+nregg2019, mm2019+aim2019, mm2019, aim2019, regg2019+nregg2019, regg2019, nregg2019],
        ['','2020', mm2020+aim2020+regg2020+nregg2020, mm2020+aim2020, mm2020, aim2020, regg2020+nregg2020, regg2020, nregg2020],
        ['','Subtotal:', mm2016+aim2016+regg2016+nregg2016+mm2017+aim2017+regg2017+nregg2017+mm2018+aim2018+regg2018+nregg2018+mm2019+aim2019+regg2019+nregg2019+mm2020+aim2020+regg2020+nregg2020
         , mm2016+aim2016+mm2017+aim2017+mm2018+aim2018+mm2019+aim2019+mm2020+aim2020,mm2016+mm2017+mm2018+mm2019+mm2020, aim2016+aim2017+aim2018+aim2019+aim2020,
         regg2016+nregg2016+regg2017+nregg2017+regg2018+nregg2018+regg2019+nregg2019+regg2020+nregg2020, regg2016+regg2017+regg2018+regg2019+regg2020,
         nregg2016+nregg2017+nregg2018+nregg2019+nregg2020],
        ['Total','Client years:', tobs, mm2011+aim2011+mm2012+aim2012+mm2013+aim2013+mm2014+aim2014+mm2015+aim2015+mm2016+aim2016+mm2017+aim2017+mm2018+aim2018+mm2019+aim2019+mm2020+aim2020
         , mm2016+mm2017+mm2018+mm2019+mm2020+mm2011+mm2012+mm2013+mm2014+mm2015,
         aim2016+aim2017+aim2018+aim2019+aim2020+aim2011+aim2012+aim2013+aim2014+aim2015, regg2011+nregg2011+regg2012+nregg2012+regg2013+nregg2013+regg2014+nregg2014+regg2015+nregg2015+
         regg2016+nregg2016+regg2017+nregg2017+regg2018+nregg2018+regg2019+nregg2019+regg2020+nregg2020, regg2016+regg2017+regg2018+regg2019+regg2020+
         regg2011+regg2012+regg2013+regg2014+regg2015, nregg2016+nregg2017+nregg2018+nregg2019+nregg2020+nregg2011+nregg2012+nregg2013+nregg2014+nregg2015],
        ['','Clients', ttobs, uISINmm+uISINaim, uISINmm, uISINaim, uISINregg+uISINnregg, uISINregg, uISINnregg]]

# add column names
df = pd.DataFrame(data, columns=['Period', 'Financial Year', 'Total', 'Total LSE', 'LSE MM', 'LSE AIM', 'Total FSE', 'FSE regulated', 'FSE not regulated'])

# print dataframe
print(df)

         Period Financial Year  Total  Total LSE  LSE MM  LSE AIM  Total FSE  \
0    Pre-reform           2011   1310       1016     572      444        294   
1                         2012   1354       1052     582      470        302   
2                         2013   1421       1116     603      513        305   
3                         2014   1471       1155     619      536        316   
4                         2015   1520       1202     637      565        318   
5                    Subtotal:   7076       5541    3013     2528       1535   
6   Post-reform           2016   1573       1254     667      587        319   
7                         2017   1621       1300     690      610        321   
8                         2018   1656       1332     706      626        324   
9                         2019   1640       1317     714      603        323   
10                        2020   1635       1307     699      608        328   
11                   Subtotal:   8125   

In [ ]:
#Table 3 Panel B
# specific for regulated and not regulated markets in Germany and the UK out of our sample

In [ ]:
# industries' comparison for regulated and not regulated markets
uk=pd.concat([uk_mm,uk_aim])
induk=uk["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
induk["Percentages"]=induk["counts"]/len(uk)*100
print(induk)

                                unique_values  counts  Percentages
0            Finance, Insurance & Real Estate    5285    35.723942
1                                    Services    3647    24.651886
2                               Manufacturing    2481    16.770312
3                                      Mining    1453     9.821549
4                                Retail Trade     577     3.900230
5           Transportation & Public Utilities     469     3.170204
6                                Construction     419     2.832229
7                             Wholesale Trade     349     2.359064
8  Agriculture, Forestry, Fishing and Hunting      83     0.561038
9                       Public Administration      31     0.209544


In [ ]:
germany=pd.concat([ger_reg,ger_nreg])
indger=germany["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
indger["Percentages"]=indger["counts"]/len(germany)*100
print(indger)

                                unique_values  counts  Percentages
0                               Manufacturing    1422    39.676339
1                                    Services     883    24.637277
2            Finance, Insurance & Real Estate     666    18.582589
3           Transportation & Public Utilities     207     5.775670
4                             Wholesale Trade     182     5.078125
5                                Retail Trade     112     3.125000
6                                Construction      49     1.367188
7  Agriculture, Forestry, Fishing and Hunting      33     0.920759
8                                      Mining      30     0.837054


In [ ]:
indmm=uk_mm["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
indmm["Percentages"]=indmm["counts"]/len(uk_mm)*100
print(indmm)

                                unique_values  counts  Percentages
0            Finance, Insurance & Real Estate    4280    53.473263
1                                    Services    1210    15.117441
2                               Manufacturing    1066    13.318341
3                                      Mining     385     4.810095
4                                Retail Trade     381     4.760120
5           Transportation & Public Utilities     284     3.548226
6                                Construction     268     3.348326
7                             Wholesale Trade      89     1.111944
8  Agriculture, Forestry, Fishing and Hunting      37     0.462269
9                       Public Administration       4     0.049975


In [ ]:
indaim=uk_aim["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
indaim["Percentages"]=indaim["counts"]/len(uk_aim)*100
print(indaim)

                                unique_values  counts  Percentages
0                                    Services    2437    35.891016
1                               Manufacturing    1415    20.839470
2                                      Mining    1068    15.729013
3            Finance, Insurance & Real Estate    1005    14.801178
4                             Wholesale Trade     260     3.829161
5                                Retail Trade     196     2.886598
6           Transportation & Public Utilities     185     2.724595
7                                Construction     151     2.223859
8  Agriculture, Forestry, Fishing and Hunting      46     0.677467
9                       Public Administration      27     0.397644


In [ ]:
indgerr=ger_reg["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
indgerr["Percentages"]=indgerr["counts"]/len(ger_reg)*100
print(indgerr)

                                unique_values  counts  Percentages
0                               Manufacturing    1223    44.183526
1                                    Services     720    26.011561
2            Finance, Insurance & Real Estate     390    14.089595
3           Transportation & Public Utilities     132     4.768786
4                             Wholesale Trade     126     4.552023
5                                Retail Trade     102     3.684971
6                                Construction      32     1.156069
7  Agriculture, Forestry, Fishing and Hunting      23     0.830925
8                                      Mining      20     0.722543


In [ ]:
indgernr=ger_nreg["Industry"].value_counts().rename_axis('unique_values').reset_index(name='counts')
indgernr["Percentages"]=indgernr["counts"]/len(ger_nreg)*100
print(indgernr)

                                unique_values  counts  Percentages
0            Finance, Insurance & Real Estate     276    33.823529
1                               Manufacturing     199    24.387255
2                                    Services     163    19.975490
3           Transportation & Public Utilities      75     9.191176
4                             Wholesale Trade      56     6.862745
5                                Construction      17     2.083333
6                                      Mining      10     1.225490
7                                Retail Trade      10     1.225490
8  Agriculture, Forestry, Fishing and Hunting      10     1.225490


In [ ]:
#Table 3 Panel C
#LSE MM
# For all: aggregate mean on concatenated value, receive mean, then calculate median and mean, as prescribed in the paper
#at
uk_mmfin = pd.merge(uk_mm, comp, how="inner", on="yearISIN")
# procure client count
uk_mmfin_c=len(uk_mmfin["ISIN"].unique())

uk_mmfina=uk_mmfin.groupby(['ISIN'], as_index=False).agg(
                      {'at':['mean',"median"]})
uk_mmfina.columns = ['ISIN','Mean',"Median"]
uk_mmfina=uk_mmfina.reindex(columns=sorted(uk_mmfina.columns))
uk_mmfinam=uk_mmfina["Mean"].mean()
uk_mmfinam2=uk_mmfina["Mean"].median()

#leverage
uk_mmfinl=uk_mmfin.groupby(['ISIN'], as_index=False).agg(
                      {'leverage':['mean',"median"]})
uk_mmfinl.columns = ['ISIN','Mean',"Median"]
uk_mmfinl=uk_mmfinl.reindex(columns=sorted(uk_mmfinl.columns))
uk_mmfinlm=uk_mmfinl["Mean"].mean()*100
uk_mmfinlm2=uk_mmfinl["Mean"].median()*100

#roa

uk_mmfinr=uk_mmfin.groupby(['ISIN'], as_index=False).agg(
                      {'roa':['mean',"median"]})
uk_mmfinr.columns = ['ISIN','Mean',"Median"]
uk_mmfinr=uk_mmfinr.reindex(columns=sorted(uk_mmfinr.columns))
uk_mmfinrm=uk_mmfinr["Mean"].mean()*100
uk_mmfinrm2=uk_mmfinr["Mean"].median()*100

#loss

uk_mmfinlo=uk_mmfin.groupby(['ISIN'], as_index=False).agg(
                      {'loss':['mean',"median"]})
uk_mmfinlo.columns = ['ISIN','Mean',"Median"]
uk_mmfinlo=uk_mmfinlo.reindex(columns=sorted(uk_mmfinlo.columns))
uk_mmfinlom=uk_mmfinlo["Mean"].mean()*100
uk_mmfinlom2=uk_mmfinlo["Mean"].median()*100

#LSE AIM
#at
uk_aimfin = pd.merge(uk_aim, comp, how="inner", on="yearISIN")
# procure client count
uk_aimfin_c=len(uk_aimfin["ISIN"].unique())

uk_aimfina=uk_aimfin.groupby(['ISIN'], as_index=False).agg(
                      {'at':['mean',"median"]})
uk_aimfina.columns = ['ISIN','Mean',"Median"]
uk_aimfina=uk_aimfina.reindex(columns=sorted(uk_aimfina.columns))
uk_aimfinam=uk_aimfina["Mean"].mean()
uk_aimfinam2=uk_aimfina["Mean"].median()

#leverage
uk_aimfinl=uk_aimfin.groupby(['ISIN'], as_index=False).agg(
                      {'leverage':['mean',"median"]})
uk_aimfinl.columns = ['ISIN','Mean',"Median"]
uk_aimfinl=uk_aimfinl.reindex(columns=sorted(uk_aimfinl.columns))
uk_aimfinlm=uk_aimfinl["Mean"].mean()*100
uk_aimfinlm2=uk_aimfinl["Mean"].median()*100

#roa

uk_aimfinr=uk_aimfin.groupby(['ISIN'], as_index=False).agg(
                      {'roa':['mean',"median"]})
uk_aimfinr.columns = ['ISIN','Mean',"Median"]
uk_aimfinr=uk_aimfinr.reindex(columns=sorted(uk_aimfinr.columns))
uk_aimfinrm=uk_aimfinr["Mean"].mean()*100
uk_aimfinrm2=uk_aimfinr["Mean"].median()*100

#loss

uk_aimfinlo=uk_aimfin.groupby(['ISIN'], as_index=False).agg(
                      {'loss':['mean',"median"]})
uk_aimfinlo.columns = ['ISIN','Mean',"Median"]
uk_aimfinlo=uk_aimfinlo.reindex(columns=sorted(uk_aimfinlo.columns))
uk_aimfinlom=uk_aimfinlo["Mean"].mean()*100
uk_aimfinlom2=uk_aimfinlo["Mean"].median()*100

#Germany regulated
#at
ger_regfin = pd.merge(ger_reg, comp, how="inner", on="yearISIN")
# procure client count
ger_regfin_c=len(ger_regfin["ISIN"].unique())

ger_regfina=ger_regfin.groupby(['ISIN'], as_index=False).agg(
                      {'at':['mean',"median"]})
ger_regfina.columns = ['ISIN','Mean',"Median"]
ger_regfina=ger_regfina.reindex(columns=sorted(ger_regfina.columns))
ger_regfinam=ger_regfina["Mean"].mean()
ger_regfinam2=ger_regfina["Mean"].median()

#leverage
ger_regfinl=ger_regfin.groupby(['ISIN'], as_index=False).agg(
                      {'leverage':['mean',"median"]})
ger_regfinl.columns = ['ISIN','Mean',"Median"]
ger_regfinl=ger_regfinl.reindex(columns=sorted(ger_regfinl.columns))
ger_regfinlm=ger_regfinl["Mean"].mean()*100
ger_regfinlm2=ger_regfinl["Mean"].median()*100

#roa

ger_regfinr=ger_regfin.groupby(['ISIN'], as_index=False).agg(
                      {'roa':['mean',"median"]})
ger_regfinr.columns = ['ISIN','Mean',"Median"]
ger_regfinr=ger_regfinr.reindex(columns=sorted(ger_regfinr.columns))
ger_regfinrm=ger_regfinr["Mean"].mean()*100
ger_regfinrm2=ger_regfinr["Mean"].median()*100

#loss

ger_regfinlo=ger_regfin.groupby(['ISIN'], as_index=False).agg(
                      {'loss':['mean',"median"]})
ger_regfinlo.columns = ['ISIN','Mean',"Median"]
ger_regfinlo=ger_regfinlo.reindex(columns=sorted(ger_regfinlo.columns))
ger_regfinlom=ger_regfinlo["Mean"].mean()*100
ger_regfinlom2=ger_regfinlo["Mean"].median()*100

#Germany not regulated
#at
ger_nregfin = pd.merge(ger_nreg, comp, how="inner", on="yearISIN")
# procure client count
ger_nregfin_c=len(ger_nregfin["ISIN"].unique())

ger_nregfina=ger_nregfin.groupby(['ISIN'], as_index=False).agg(
                      {'at':['mean',"median"]})
ger_nregfina.columns = ['ISIN','Mean',"Median"]
ger_nregfina=ger_nregfina.reindex(columns=sorted(ger_nregfina.columns))
ger_nregfinam=ger_nregfina["Mean"].mean()
ger_nregfinam2=ger_nregfina["Mean"].median()

#leverage
ger_nregfinl=ger_nregfin.groupby(['ISIN'], as_index=False).agg(
                      {'leverage':['mean',"median"]})
ger_nregfinl.columns = ['ISIN','Mean',"Median"]
ger_nregfinl=ger_nregfinl.reindex(columns=sorted(ger_nregfinl.columns))
ger_nregfinlm=ger_nregfinl["Mean"].mean()*100
ger_nregfinlm2=ger_nregfinl["Mean"].median()*100

#roa

ger_nregfinr=ger_nregfin.groupby(['ISIN'], as_index=False).agg(
                      {'roa':['mean',"median"]})
ger_nregfinr.columns = ['ISIN','Mean',"Median"]
ger_nregfinr=ger_nregfinr.reindex(columns=sorted(ger_nregfinr.columns))
ger_nregfinrm=ger_nregfinr["Mean"].mean()*100
ger_nregfinrm2=ger_nregfinr["Mean"].median()*100

#loss

ger_nregfinlo=ger_nregfin.groupby(['ISIN'], as_index=False).agg(
                      {'loss':['mean',"median"]})
ger_nregfinlo.columns = ['ISIN','Mean',"Median"]
ger_nregfinlo=ger_nregfinlo.reindex(columns=sorted(ger_nregfinlo.columns))
ger_nregfinlom=ger_nregfinlo["Mean"].mean()*100
ger_nregfinlom2=ger_nregfinlo["Mean"].median()*100



# build dataframe
# insert observation data
data = [['Total assets (in m of euros)', "Mean",uk_mmfinam,uk_aimfinam,ger_regfinam,ger_nregfinam]
        ,['', "Median",uk_mmfinam2,uk_aimfinam2,ger_regfinam2,ger_nregfinam2]
        ,['Leverage %', "Mean",uk_mmfinlm,uk_aimfinlm,ger_regfinlm,ger_nregfinlm]
        ,['', "Median",uk_mmfinlm2,uk_aimfinlm2,ger_regfinlm2,ger_nregfinlm2]
        ,['Return on assets %', "Mean",uk_mmfinrm,uk_aimfinrm,ger_regfinrm,ger_nregfinrm]
        ,['', "Median",uk_mmfinrm2,uk_aimfinrm2,ger_regfinrm2,ger_nregfinrm2]
        ,['Loss %', "Mean",uk_mmfinlom,uk_aimfinlom,ger_regfinlom,ger_nregfinlom]
        ,['', "Median",uk_mmfinlom2,uk_aimfinlom2,ger_regfinlom2,ger_nregfinlom2]
        ,['Total', "Clients",uk_mmfin_c,uk_aimfin_c,ger_regfin_c,ger_nregfin_c]]

# Create the dataframe
df = pd.DataFrame(data, columns=['Variable', 'Metric','LSE MM', 'LSE AIM','FSE regulated', 'FSE not regulated'])

# print dataframe.
print(df)

                       Variable   Metric        LSE MM     LSE AIM  \
0  Total assets (in m of euros)     Mean  16594.858668   76.042021   
1                                 Median    397.833333   26.109333   
2                    Leverage %     Mean    119.718316   60.348795   
3                                 Median     38.809673   38.226210   
4            Return on assets %     Mean    -12.739858  -23.306976   
5                                 Median      4.660800   -4.336751   
6                        Loss %     Mean     21.767165   50.932625   
7                                 Median     14.285714   50.000000   
8                         Total  Clients    691.000000  653.000000   

   FSE regulated  FSE not regulated  
0   24702.865621        1738.631060  
1     579.630542          80.330167  
2      57.952611          53.877988  
3      57.437902          56.711005  
4       1.660741           0.985720  
5       3.060299           2.128365  
6      20.496416          29.6385

In [ ]:
# counts
print(uk_mmfin_c)
print(uk_aimfin_c)
print(ger_regfin_c)
print(ger_nregfin_c)

691
653
248
87


In [ ]:
#Table 5 Panel A1, B1

In [ ]:
# create pre- and post-reform datasets
uk_mm_pre=uk_mm[(uk_mm["Year"] == "2011") | (uk_mm["Year"] == "2012")| (uk_mm["Year"] == "2013")| (uk_mm["Year"] == "2014")| (uk_mm["Year"] == "2015")]
uk_mm_post=uk_mm[(uk_mm["Year"] == "2016") | (uk_mm["Year"] == "2017")| (uk_mm["Year"] == "2018")| (uk_mm["Year"] == "2019")| (uk_mm["Year"] == "2020")]

ger_reg_pre=ger_reg[(ger_reg["Year"] == "2011") | (ger_reg["Year"] == "2012")| (ger_reg["Year"] == "2013")| (ger_reg["Year"] == "2014")| (ger_reg["Year"] == "2015")]
ger_reg_post=ger_reg[(ger_reg["Year"] == "2016") | (ger_reg["Year"] == "2017")| (ger_reg["Year"] == "2018")| (ger_reg["Year"] == "2019")| (ger_reg["Year"] == "2020")]

In [ ]:
# total sums for fees market share and counts for client market share
agfees_uk_pre=uk_mm_pre['AUDIT_FEES_EUR'].sum()
print(agfees_uk_pre)
agfees_uk_post=uk_mm_post['AUDIT_FEES_EUR'].sum()
print(agfees_uk_post)

agfees_ger_pre=ger_reg_pre['AUDIT_FEES_EUR'].sum()
print(agfees_ger_pre)
agfees_ger_post=ger_reg_post['AUDIT_FEES_EUR'].sum()
print(agfees_ger_post)

agcl_uk_pre=uk_mm_pre['AUDITOR_NETWORK'].count()
agcl_uk_post=uk_mm_post['AUDITOR_NETWORK'].count()

agcl_ger_pre=ger_reg_pre['AUDITOR_NETWORK'].count()
agcl_ger_post=ger_reg_post['AUDITOR_NETWORK'].count()

4882004989.0
6142211093.0
2985356836.0
3575187118.0


In [ ]:
# total auditing firms
uk_mm_pre_audc=len(uk_mm_pre['AUDITOR_NETWORK'].unique())
print(uk_mm_pre_audc)
uk_mm_post_audc=len(uk_mm_post['AUDITOR_NETWORK'].unique())
print(uk_mm_post_audc)
ger_reg_pre_audc=len(ger_reg_pre['AUDITOR_NETWORK'].unique())
print(ger_reg_pre_audc)
ger_reg_post_audc=len(ger_reg_post['AUDITOR_NETWORK'].unique())
print(ger_reg_post_audc)

32
37
40
32


In [ ]:
# individual aggreggated sums and means for fees market share and counts for client market share
uk_mm_pre_anet=uk_mm_pre.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDIT_FEES_EUR':['sum',"mean"]})
uk_mm_pre_anet.columns = ['Name','Sum',"Average"]
uk_mm_pre_anet=uk_mm_pre_anet.reindex(columns=sorted(uk_mm_pre_anet.columns))

uk_mm_post_anet=uk_mm_post.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDIT_FEES_EUR':['sum',"mean"]})
uk_mm_post_anet.columns = ['Name','Sum',"Average"]
uk_mm_post_anet=uk_mm_post_anet.reindex(columns=sorted(uk_mm_post_anet.columns))

ger_reg_pre_anet=ger_reg_pre.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDIT_FEES_EUR':['sum',"mean"]})
ger_reg_pre_anet.columns = ['Name','Sum',"Average"]
ger_reg_pre_anet=ger_reg_pre_anet.reindex(columns=sorted(ger_reg_pre_anet.columns))

ger_reg_post_anet=ger_reg_post.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDIT_FEES_EUR':['sum',"mean"]})
ger_reg_post_anet.columns = ['Name','Sum',"Average"]
ger_reg_post_anet=ger_reg_post_anet.reindex(columns=sorted(ger_reg_post_anet.columns))

In [ ]:
uk_mm_pre_cl=uk_mm_pre.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDITOR_NETWORK':['first','count']})
uk_mm_pre_cl.columns = ['Name','Count']
uk_mm_pre_cl=uk_mm_pre_cl.reindex(columns=sorted(uk_mm_pre_cl.columns))

uk_mm_post_cl=uk_mm_post.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDITOR_NETWORK':['first','count']})
uk_mm_post_cl.columns = ['Name','Count']
uk_mm_post_cl=uk_mm_post_cl.reindex(columns=sorted(uk_mm_post_cl.columns))

ger_reg_pre_cl=ger_reg_pre.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDITOR_NETWORK':['first','count']})
ger_reg_pre_cl.columns = ['Name','Count']
ger_reg_pre_cl=ger_reg_pre_cl.reindex(columns=sorted(ger_reg_pre_cl.columns))

ger_reg_post_cl=ger_reg_post.groupby(['AUDITOR_NETWORK'], as_index=False).agg(
                      {'AUDITOR_NETWORK':['first','count']})
ger_reg_post_cl.columns = ['Name','Count']
ger_reg_post_cl=ger_reg_post_cl.reindex(columns=sorted(ger_reg_post_cl.columns))

In [ ]:
# market concentration calculation (market share)
uk_mm_pre_anet["Market Concentration (Fees)"]=uk_mm_pre_anet["Sum"]/agfees_uk_pre*100
uk_mm_post_anet["Market Concentration (Fees)"]=uk_mm_post_anet["Sum"]/agfees_uk_post*100

ger_reg_pre_anet["Market Concentration (Fees)"]=ger_reg_pre_anet["Sum"]/agfees_ger_pre*100
ger_reg_post_anet["Market Concentration (Fees)"]=ger_reg_post_anet["Sum"]/agfees_ger_post*100

uk_mm_pre_cl["Market Concentration (Clients)"]=uk_mm_pre_cl["Count"]/agcl_uk_pre*100
uk_mm_post_cl["Market Concentration (Clients)"]=uk_mm_post_cl["Count"]/agcl_uk_post*100

ger_reg_pre_cl["Market Concentration (Clients)"]=ger_reg_pre_cl["Count"]/agcl_ger_pre*100
ger_reg_post_cl["Market Concentration (Clients)"]=ger_reg_post_cl["Count"]/agcl_ger_post*100

In [ ]:
# HHI All
# Squared market shares
uk_mm_pre_anet["Squared share"]=uk_mm_pre_anet["Market Concentration (Fees)"]**2
uk_mm_post_anet["Squared share"]=uk_mm_post_anet["Market Concentration (Fees)"]**2

ger_reg_pre_anet["Squared share"]=ger_reg_pre_anet["Market Concentration (Fees)"]**2
ger_reg_post_anet["Squared share"]=ger_reg_post_anet["Market Concentration (Fees)"]**2

uk_mm_pre_cl["Squared share"]=uk_mm_pre_cl["Market Concentration (Clients)"]**2
uk_mm_post_cl["Squared share"]=uk_mm_post_cl["Market Concentration (Clients)"]**2

ger_reg_pre_cl["Squared share"]=ger_reg_pre_cl["Market Concentration (Clients)"]**2
ger_reg_post_cl["Squared share"]=ger_reg_post_cl["Market Concentration (Clients)"]**2

#Sum to receive HHI values
uk_mm_pre_anetHHIall=uk_mm_pre_anet["Squared share"].sum()
uk_mm_post_anetHHIall=uk_mm_post_anet["Squared share"].sum()

ger_reg_pre_anetHHIall=ger_reg_pre_anet["Squared share"].sum()
ger_reg_post_anetHHIall=ger_reg_post_anet["Squared share"].sum()

uk_mm_pre_clHHIall=uk_mm_pre_cl["Squared share"].sum()
uk_mm_post_clHHIall=uk_mm_post_cl["Squared share"].sum()

ger_reg_pre_clHHIall=ger_reg_pre_cl["Squared share"].sum()
ger_reg_post_clHHIall=ger_reg_post_cl["Squared share"].sum()

In [ ]:
# create a binary variable to distinguish the firms from the table
uk_mm_pre_anet["Included Network"]=uk_mm_pre_anet.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

uk_mm_pre_cl["Included Network"]=uk_mm_pre_cl.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

In [ ]:
uk_mm_post_anet["Included Network"]=uk_mm_post_anet.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

uk_mm_post_cl["Included Network"]=uk_mm_post_cl.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

In [ ]:
ger_reg_pre_anet["Included Network"]=ger_reg_pre_anet.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

ger_reg_pre_cl["Included Network"]=ger_reg_pre_cl.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

In [ ]:
ger_reg_post_anet["Included Network"]=ger_reg_post_anet.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

ger_reg_post_cl["Included Network"]=ger_reg_post_cl.apply(lambda x : 0 if x['Name'] == "|PricewaterhouseCoopers International|"
                                      or x['Name'] == "|Nexia International|"
                                      or x['Name'] == "|KPMG International|"
                                      or x['Name'] == "|Grant Thornton International|"
                                      or x['Name'] == "|Ernst & Young Global|"
                                      or x['Name'] == "|Deloitte & Touche International|"
                                      or x['Name'] == "|BDO International|"
                                      or x['Name'] == "|Baker Tilly International|" else 1, axis=1)

In [ ]:
# select only networks from the paper and sort descending
uk_mm_pre_anet=uk_mm_pre_anet[uk_mm_pre_anet['Included Network']==0]
uk_mm_pre_anet=uk_mm_pre_anet.sort_values(by=['Market Concentration (Fees)'],ascending=0)

uk_mm_post_anet=uk_mm_post_anet[uk_mm_post_anet['Included Network']==0]
uk_mm_post_anet=uk_mm_post_anet.sort_values(by=['Market Concentration (Fees)'],ascending=0)

ger_reg_pre_anet=ger_reg_pre_anet[ger_reg_pre_anet['Included Network']==0]
ger_reg_pre_anet=ger_reg_pre_anet.sort_values(by=['Market Concentration (Fees)'],ascending=0)

ger_reg_post_anet=ger_reg_post_anet[ger_reg_post_anet['Included Network']==0]
ger_reg_post_anet=ger_reg_post_anet.sort_values(by=['Market Concentration (Fees)'],ascending=0)
####
uk_mm_pre_cl=uk_mm_pre_cl[uk_mm_pre_cl['Included Network']==0]
uk_mm_pre_cl=uk_mm_pre_cl.sort_values(by=['Market Concentration (Clients)'],ascending=0)

uk_mm_post_cl=uk_mm_post_cl[uk_mm_post_cl['Included Network']==0]
uk_mm_post_cl=uk_mm_post_cl.sort_values(by=['Market Concentration (Clients)'],ascending=0)

ger_reg_pre_cl=ger_reg_pre_cl[ger_reg_pre_cl['Included Network']==0]
ger_reg_pre_cl=ger_reg_pre_cl.sort_values(by=['Market Concentration (Clients)'],ascending=0)

ger_reg_post_cl=ger_reg_post_cl[ger_reg_post_cl['Included Network']==0]
ger_reg_post_cl=ger_reg_post_cl.sort_values(by=['Market Concentration (Clients)'],ascending=0)

In [ ]:
# delete some unnecessary columns
del uk_mm_pre_anet["Included Network"]
del uk_mm_pre_cl["Included Network"]
del uk_mm_post_anet["Included Network"]
del uk_mm_post_cl["Included Network"]

del ger_reg_pre_anet["Included Network"]
del ger_reg_pre_cl["Included Network"]
del ger_reg_post_anet["Included Network"]
del ger_reg_post_cl["Included Network"]

In [ ]:
# replace names for readability purposes
uk_mm_pre_anet=uk_mm_pre_anet.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])

uk_mm_post_anet=uk_mm_post_anet.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])

uk_mm_pre_cl=uk_mm_pre_cl.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])

uk_mm_post_cl=uk_mm_post_cl.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])
#######
ger_reg_pre_anet=ger_reg_pre_anet.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])

ger_reg_post_anet=ger_reg_post_anet.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])

ger_reg_pre_cl=ger_reg_pre_cl.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])

ger_reg_post_cl=ger_reg_post_cl.replace(["|KPMG International|","|Deloitte & Touche International|","|Ernst & Young Global|"
,"|Grant Thornton International|","|BDO International|"
,"|Nexia International|","|Baker Tilly International|","|PricewaterhouseCoopers International|"]
, ["KPMG","Deloitte","EY","Grant Thornton","BDO","Nexia","Baker Tilly","PwC"])


In [ ]:
# create a binary variable for the Big4 identification
uk_mm_pre_cl["Big 4"]=uk_mm_pre_cl.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)
uk_mm_post_cl["Big 4"]=uk_mm_post_cl.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)

ger_reg_pre_cl["Big 4"]=ger_reg_pre_cl.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)
ger_reg_post_cl["Big 4"]=ger_reg_post_cl.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)

uk_mm_pre_anet["Big 4"]=uk_mm_pre_anet.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)
uk_mm_post_anet["Big 4"]=uk_mm_post_anet.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)

ger_reg_pre_anet["Big 4"]=ger_reg_pre_anet.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)
ger_reg_post_anet["Big 4"]=ger_reg_post_anet.apply(lambda x : 0 if x['Name'] == "PwC"
                                      or x['Name'] == "Deloitte"
                                      or x['Name'] == "EY"
                                      or x['Name'] == "KPMG" else 1, axis=1)

In [ ]:
# calculate Big 4 market share
uk_mm_pre_cl4=uk_mm_pre_cl[uk_mm_pre_cl['Big 4']==0]["Market Concentration (Clients)"].sum()
uk_mm_post_cl4=uk_mm_post_cl[uk_mm_post_cl['Big 4']==0]["Market Concentration (Clients)"].sum()
ger_reg_pre_cl4=ger_reg_pre_cl[ger_reg_pre_cl['Big 4']==0]["Market Concentration (Clients)"].sum()
ger_reg_post_cl4=ger_reg_post_cl[ger_reg_post_cl['Big 4']==0]["Market Concentration (Clients)"].sum()

uk_mm_pre_anet4=uk_mm_pre_anet[uk_mm_pre_anet['Big 4']==0]["Market Concentration (Fees)"].sum()
uk_mm_post_anet4=uk_mm_post_anet[uk_mm_post_anet['Big 4']==0]["Market Concentration (Fees)"].sum()
ger_reg_pre_anet4=ger_reg_pre_anet[ger_reg_pre_anet['Big 4']==0]["Market Concentration (Fees)"].sum()
ger_reg_post_anet4=ger_reg_post_anet[ger_reg_post_anet['Big 4']==0]["Market Concentration (Fees)"].sum()

# calculate Big4 HHI
uk_mm_pre_anetHHI4=uk_mm_pre_anet[uk_mm_pre_anet['Big 4']==0]["Squared share"].sum()
uk_mm_post_anetHHI4=uk_mm_post_anet[uk_mm_post_anet['Big 4']==0]["Squared share"].sum()

ger_reg_pre_anetHHI4=ger_reg_pre_anet[ger_reg_pre_anet['Big 4']==0]["Squared share"].sum()
ger_reg_post_anetHHI4=ger_reg_post_anet[ger_reg_post_anet['Big 4']==0]["Squared share"].sum()

uk_mm_pre_clHHI4=uk_mm_pre_cl[uk_mm_pre_cl['Big 4']==0]["Squared share"].sum()
uk_mm_post_clHHI4=uk_mm_post_cl[uk_mm_post_cl['Big 4']==0]["Squared share"].sum()

ger_reg_pre_clHHI4=ger_reg_pre_cl[ger_reg_pre_cl['Big 4']==0]["Squared share"].sum()
ger_reg_post_clHHI4=ger_reg_post_cl[ger_reg_post_cl['Big 4']==0]["Squared share"].sum()

In [ ]:
# calculate largest 4 share
uk_mm_pre_cll4=uk_mm_pre_cl.head(n=4)["Market Concentration (Clients)"].sum()
uk_mm_post_cll4=uk_mm_post_cl.head(n=4)["Market Concentration (Clients)"].sum()
ger_reg_pre_cll4=ger_reg_pre_cl.head(n=4)["Market Concentration (Clients)"].sum()
ger_reg_post_cll4=ger_reg_post_cl.head(n=4)["Market Concentration (Clients)"].sum()

uk_mm_pre_anetl4=uk_mm_pre_anet.head(n=4)["Market Concentration (Fees)"].sum()
uk_mm_post_anetl4=uk_mm_post_anet.head(n=4)["Market Concentration (Fees)"].sum()
ger_reg_pre_anetl4=ger_reg_pre_anet.head(n=4)["Market Concentration (Fees)"].sum()
ger_reg_post_anetl4=ger_reg_post_anet.head(n=4)["Market Concentration (Fees)"].sum()

In [ ]:
#Panel A2, B2

In [ ]:
# merge on name, we need merged to calculate average fees
uk_mm_pre_afpc=pd.merge(uk_mm_pre_anet, uk_mm_pre_cl, on="Name")
uk_mm_post_afpc=pd.merge(uk_mm_post_anet, uk_mm_post_cl, on="Name")

ger_reg_pre_afpc=pd.merge(ger_reg_pre_anet, ger_reg_pre_cl, on="Name")
ger_reg_post_afpc=pd.merge(ger_reg_post_anet, ger_reg_post_cl, on="Name")

In [ ]:
# calculate average fees and round on two decimal points for better presentation
uk_mm_pre_afpc["Average Audit Fees per Client"]=round(uk_mm_pre_afpc["Sum"]/uk_mm_pre_afpc["Count"],2)
uk_mm_post_afpc["Average Audit Fees per Client"]=round(uk_mm_post_afpc["Sum"]/uk_mm_post_afpc["Count"],2)

ger_reg_pre_afpc["Average Audit Fees per Client"]=round(ger_reg_pre_afpc["Sum"]/ger_reg_pre_afpc["Count"],2)
ger_reg_post_afpc["Average Audit Fees per Client"]=round(ger_reg_post_afpc["Sum"]/ger_reg_post_afpc["Count"],2)

In [ ]:
# Big 4 average fees based on the binary variable produced above
uk_mm_pre_afpc4=uk_mm_pre_afpc[uk_mm_pre_afpc['Big 4_x']==0]["Average Audit Fees per Client"].sum()/4
print(uk_mm_pre_afpc4)
uk_mm_post_afpc4=uk_mm_post_afpc[uk_mm_post_afpc['Big 4_x']==0]["Average Audit Fees per Client"].sum()/4
print(uk_mm_post_afpc4)
ger_reg_pre_afpc4=ger_reg_pre_afpc[ger_reg_pre_afpc['Big 4_x']==0]["Average Audit Fees per Client"].sum()/4
print(ger_reg_pre_afpc4)
ger_reg_post_afpc4=ger_reg_post_afpc[ger_reg_post_afpc['Big 4_x']==0]["Average Audit Fees per Client"].sum()/4
print(ger_reg_post_afpc4)

1544410.2125
1894847.4775
2473407.185
3072218.1225000005
